In [ ]:
import pandas as pd
import numpy as np

import os
import sys
import cv2
import re

import seaborn as sns
import torch
import requests
import pickle

from google.colab.patches import cv2_imshow

from PIL import Image
from io import BytesIO

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import HTML,display, Javascript,HTML, Audio, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import base64
from base64 import b64decode
from moviepy.editor import VideoFileClip

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import keras
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from keras.models import Sequential,load_model
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.utils.np_utils import to_categorical

import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop

In [ ]:
observed_emotions=['angry', 'disgust', 'fear', 'happy','neutral','sad','surprise']

# RAVDESS

In [ ]:
!gdown 13NCkIfuEJ-rgEQbqPo4CI31aBUQ2Mc77
!unzip "/content/archive (1).zip" -d "/content/ravdess_dataset/"

Downloading...
From: https://drive.google.com/uc?id=13NCkIfuEJ-rgEQbqPo4CI31aBUQ2Mc77
To: /content/archive (1).zip
100% 450M/450M [00:06<00:00, 64.4MB/s]
Archive:  /content/archive (1).zip
  inflating: /content/ravdess_dataset/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-02-02-01-01-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-02-02-01-02-01.wav  
  inflating: /content/ravdess_dataset/Actor_01/03-01-02-02-02-01-01.wav

In [ ]:
path = "/content/ravdess_dataset/audio_speech_actors_01-24"
ravdess_directory_list = os.listdir(path)

file_emotion = []
file_path = []

for dir in ravdess_directory_list:
    actor = os.listdir(path + "/"+ dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        file_emotion.append(int(part[2]))
        file_path.append(path + '/' + dir + '/' + file)

emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head(7)

,Emotions,Path
0,surprise,/content/ravdess_dataset/audio_speech_actors_0...
1,surprise,/content/ravdess_dataset/audio_speech_actors_0...
2,surprise,/content/ravdess_dataset/audio_speech_actors_0...
3,happy,/content/ravdess_dataset/audio_speech_actors_0...
4,happy,/content/ravdess_dataset/audio_speech_actors_0...
5,sad,/content/ravdess_dataset/audio_speech_actors_0...
6,sad,/content/ravdess_dataset/audio_speech_actors_0...


# CREMA D

In [ ]:
!gdown 1kBufqeE_ZnLAtGPZlx8E6oLnjSZ9kJft
!unzip "/content/Crema.zip" -d "/content/crema_dataset/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/crema_dataset/AudioWAV/1031_DFA_DIS_XX.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_DFA_FEA_XX.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_DFA_HAP_XX.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_DFA_NEU_XX.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_DFA_SAD_XX.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_ANG_HI.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_ANG_LO.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_ANG_MD.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_DIS_HI.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_DIS_LO.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_DIS_MD.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_FEA_HI.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_FEA_LO.wav  
  inflating: /content/crema_dataset/AudioWAV/1031_IEO_FEA_MD.wav  
  inflating

In [ ]:
crema_d = '/content/crema_dataset/AudioWAV'
crema_directory_list = os.listdir(crema_d)

file_emotion = []
file_path = []

for file in crema_directory_list:
  file_path.append(crema_d + '/'+ file)
  part=file.split('_')
  if part[2] == 'SAD':
        file_emotion.append('sad')
  elif part[2] == 'ANG':
        file_emotion.append('angry')
  elif part[2] == 'DIS':
        file_emotion.append('disgust')
  elif part[2] == 'FEA':
        file_emotion.append('fear')
  elif part[2] == 'HAP':
        file_emotion.append('happy')
  elif part[2] == 'NEU':
        file_emotion.append('neutral')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

,Emotions,Path
0,angry,/content/crema_dataset/AudioWAV/1074_IWL_ANG_X...
1,fear,/content/crema_dataset/AudioWAV/1070_ITS_FEA_X...
2,disgust,/content/crema_dataset/AudioWAV/1090_MTI_DIS_X...
3,angry,/content/crema_dataset/AudioWAV/1055_ITH_ANG_X...
4,happy,/content/crema_dataset/AudioWAV/1059_IOM_HAP_X...


# TESS

In [ ]:
!gdown 15qAUZYPDg773V0yANdugMJbNzWKVd_gF
!unzip "/content/TESS.zip" -d "/content/tess_dataset/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_back_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_bar_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_base_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_bean_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_beg_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_bite_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_boat_angry.wav  
  inflating: /content/tess_dataset/TESS Toronto emotional speech set data/OAF_angry/OAF_bone_angry.wav  
  infl

In [ ]:
tess_d = '/content/tess_dataset/TESS Toronto emotional speech set data/'
tess_directory_list = os.listdir(tess_d)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(tess_d + dir)
    for file in directories:
      #print()
      path = file.split('.')[0]
      emo  = path.split('_')[2]

      if emo=='ps':
            file_emotion.append('surprise')
      else:
            file_emotion.append(emo)
      file_path.append(tess_d +'/'+dir+'/'+file)

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

,Emotions,Path
0,surprise,/content/tess_dataset/TESS Toronto emotional s...
1,surprise,/content/tess_dataset/TESS Toronto emotional s...
2,surprise,/content/tess_dataset/TESS Toronto emotional s...
3,surprise,/content/tess_dataset/TESS Toronto emotional s...
4,surprise,/content/tess_dataset/TESS Toronto emotional s...


# SAVEE

In [ ]:
!gdown 1PFaD1QGuoeMeg7yZWnoMiBxwW1X0S1hl
!unzip "/content/Savee.zip" -d "/content/savee_dataset/"
os.remove('/content/savee_dataset/AudioData/Info.txt')

Downloading...
From: https://drive.google.com/uc?id=1PFaD1QGuoeMeg7yZWnoMiBxwW1X0S1hl
To: /content/Savee.zip
100% 225M/225M [00:03<00:00, 67.3MB/s]
Archive:  /content/Savee.zip
  inflating: /content/savee_dataset/AudioData/DC/a01.wav  
  inflating: /content/savee_dataset/AudioData/DC/a02.wav  
  inflating: /content/savee_dataset/AudioData/DC/a03.wav  
  inflating: /content/savee_dataset/AudioData/DC/a04.wav  
  inflating: /content/savee_dataset/AudioData/DC/a05.wav  
  inflating: /content/savee_dataset/AudioData/DC/a06.wav  
  inflating: /content/savee_dataset/AudioData/DC/a07.wav  
  inflating: /content/savee_dataset/AudioData/DC/a08.wav  
  inflating: /content/savee_dataset/AudioData/DC/a09.wav  
  inflating: /content/savee_dataset/AudioData/DC/a10.wav  
  inflating: /content/savee_dataset/AudioData/DC/a11.wav  
  inflating: /content/savee_dataset/AudioData/DC/a12.wav  
  inflating: /content/savee_dataset/AudioData/DC/a13.wav  
  inflating: /content/savee_dataset/AudioData/DC/a14.wav

In [ ]:
savee_d = '/content/savee_dataset/AudioData/'
savee_directory_list = os.listdir(savee_d)

file_emotion = []
file_path = []

for dir in savee_directory_list:
    directories = os.listdir(savee_d + dir)
    for file in directories:

      file_path.append(savee_d+dir+'/'+file)
      part = file.split('.')[0]
      ele = re.findall("[a-zA-Z]+", part)
      ele = ele[0]

      if ele=='a':
        file_emotion.append('angry')
      elif ele=='d':
        file_emotion.append('disgust')
      elif ele=='f':
        file_emotion.append('fear')
      elif ele=='h':
        file_emotion.append('happy')
      elif ele=='n':
        file_emotion.append('neutral')
      elif ele=='sa':
        file_emotion.append('sad')
      else:
        file_emotion.append('surprise')

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Savee_df = pd.concat([emotion_df, path_df], axis=1)
Savee_df.head()

,Emotions,Path
0,neutral,/content/savee_dataset/AudioData/KL/n05.wav
1,sad,/content/savee_dataset/AudioData/KL/sa06.wav
2,angry,/content/savee_dataset/AudioData/KL/a04.wav
3,neutral,/content/savee_dataset/AudioData/KL/n13.wav
4,surprise,/content/savee_dataset/AudioData/KL/su12.wav


# Combined

In [ ]:
# creating Dataframe using all the 4 dataframes we created so far.
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df, Savee_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.head()

,Emotions,Path
0,surprise,/content/ravdess_dataset/audio_speech_actors_0...
1,surprise,/content/ravdess_dataset/audio_speech_actors_0...
2,surprise,/content/ravdess_dataset/audio_speech_actors_0...
3,happy,/content/ravdess_dataset/audio_speech_actors_0...
4,happy,/content/ravdess_dataset/audio_speech_actors_0...


# Code

In [ ]:
labels = {'disgust':0,'happy':1,'sad':2,'neutral':3,'fear':4,'angry':5, 'surprise':6, 'calm':7}
data_path.replace({'Emotions':labels},inplace=True)

In [ ]:
data_path

,Emotions,Path
0,6,/content/ravdess_dataset/audio_speech_actors_0...
1,6,/content/ravdess_dataset/audio_speech_actors_0...
2,6,/content/ravdess_dataset/audio_speech_actors_0...
3,1,/content/ravdess_dataset/audio_speech_actors_0...
4,1,/content/ravdess_dataset/audio_speech_actors_0...
...,...,...
475,2,/content/savee_dataset/AudioData/JE/sa09.wav
476,0,/content/savee_dataset/AudioData/JE/d10.wav
477,4,/content/savee_dataset/AudioData/JE/f06.wav
478,3,/content/savee_dataset/AudioData/JE/n20.wav


In [ ]:
num_mfcc=13
n_fft=2048
hop_length=512
SAMPLE_RATE = 22050
data = { "labels": [], "mfcc": [] }


for i in range(12162):
    data['labels'].append(data_path.iloc[i,0])
    signal, sample_rate = librosa.load(data_path.iloc[i,1], sr=SAMPLE_RATE)
    mfcc = librosa.feature.mfcc(y =signal , sr=SAMPLE_RATE , n_mfcc=13, n_fft=2048, hop_length=512)
    mfcc = mfcc.T
    data["mfcc"].append(np.asarray(mfcc))
    if i%500==0:
        print(i , ' / 12162   '  ,  ((i/12162)*100) , ' %')

0  / 12162    0.0  %
500  / 12162    4.1111659266568  %
1000  / 12162    8.2223318533136  %
1500  / 12162    12.333497779970399  %
2000  / 12162    16.4446637066272  %
2500  / 12162    20.555829633284002  %
3000  / 12162    24.666995559940798  %
3500  / 12162    28.778161486597597  %
4000  / 12162    32.8893274132544  %
4500  / 12162    37.0004933399112  %
5000  / 12162    41.111659266568005  %
5500  / 12162    45.2228251932248  %
6000  / 12162    49.333991119881595  %
6500  / 12162    53.44515704653839  %
7000  / 12162    57.55632297319519  %
7500  / 12162    61.667488899851996  %
8000  / 12162    65.7786548265088  %
8500  / 12162    69.8898207531656  %
9000  / 12162    74.0009866798224  %
9500  / 12162    78.1121526064792  %
10000  / 12162    82.22331853313601  %
10500  / 12162    86.33448445979279  %
11000  / 12162    90.4456503864496  %
11500  / 12162    94.5568163131064  %
12000  / 12162    98.66798223976319  %


In [ ]:
X = np.asarray(data['mfcc'])
y = np.asarray(data["labels"])

  X = np.asarray(data['mfcc'])



In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(X)
X.shape

(12162, 308, 13)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
print(X_train.shape,y_train.shape,X_validation.shape,y_validation.shape,X_test.shape,y_test.shape)

(8756, 308, 13) (8756,) (2189, 308, 13) (2189,) (1217, 308, 13) (1217,)


In [ ]:
def build_model(input_shape):
    model = tf.keras.Sequential()

    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(LSTM(64))

    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(LSTM(64))


    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(6, activation='softmax'))

    return model

In [ ]:
def build_model(input_shape):
    model = Sequential()

    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.3))

    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(6, activation='softmax'))

    return model


In [ ]:
# create network
input_shape = (None,13)
model = build_model(input_shape)

# compile model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 64)          19968     
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 1

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

Epoch 1/30
274/274 [==============================] - 17s 28ms/step - loss: nan - accuracy: 0.1578 - val_loss: nan - val_accuracy: 0.1631
Epoch 2/30
274/274 [==============================] - 6s 22ms/step - loss: nan - accuracy: 0.1577 - val_loss: nan - val_accuracy: 0.1631
Epoch 3/30
274/274 [==============================] - 7s 24ms/step - loss: nan - accuracy: 0.1577 - val_loss: nan - val_accuracy: 0.1631
Epoch 4/30
274/274 [==============================] - 6s 21ms/step - loss: nan - accuracy: 0.1577 - val_loss: nan - val_accuracy: 0.1631
Epoch 5/30
274/274 [==============================] - 7s 25ms/step - loss: nan - accuracy: 0.1577 - val_loss: nan - val_accuracy: 0.1631
Epoch 6/30
274/274 [==============================] - 7s 25ms/step - loss: nan - accuracy: 0.1577 - val_loss: nan - val_accuracy: 0.1631
Epoch 7/30
274/274 [==============================] - 7s 25ms/step - loss: nan - accuracy: 0.1577 - val_loss: nan - val_accuracy: 0.1631
Epoch 8/30
274/274 [====================

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy: ",test_acc)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model.save('/content/gdrive/MyDrive/Projects/CSE 4622 ML/model\\emotion_detection_from_audio_last_version.h5')

In [ ]:
model.save('Speech-Emotion-Recognition-Model.h5')